Alex Medina | Problem Set #2

**Problem 1:**  

- Continuing with building a integrator framework, we implement Simpon's rule to repeat problem 3 from last set and compute $\int_{0}^{\frac{\pi}{2}} \sin(x)\, dx$.

In [ ]:
#Imports

import math as m
import numpy as np
import matplotlib.pyplot as plt

#Extra for this set
import random

In [2]:
#For consideration, lets rewrite the rectangle integration from pset1, but cleaning it up

def rect_int(fct, a, b, N):
    #fct = function we are integration
    #lower and upper limits a and b
    #number of steps N

    #width deltax 
    delx = (b -a)/N
    
    result = 0
    for i in np.arange(N):
        x = a + i * delx
        result += fct(x) * delx
    return result

In [3]:
#We want to try it with sin again

def sin_test(x):
    return np.sin(x)

In [4]:
for i in range(1,6):
    lower = 0
    upper = np.pi/2
    numb_x = 10**i
    integral = rect_int(sin_test, lower, upper, numb_x)
    print(f"N = {i};", f"integral = {integral};", f"error = {abs(integral - 1.0)/1.0}")

N = 1; integral = 0.9194031700146124; error = 0.08059682998538764
N = 2; integral = 0.9921254566056334; error = 0.007874543394366551
N = 3; integral = 0.9992143962198363; error = 0.0007856037801636795
N = 4; integral = 0.9999214581274958; error = 7.854187250422306e-05
N = 5; integral = 0.9999921459978001; error = 7.85400219993626e-06


As expected, the error decreases as 1/N.

In [5]:
#Now trying the Simpson integrator

def simpson_int(fct, a, b, N):
    #Goes as (1/6 f0 + 2/3 f0.5 + 1/3 f1 + 2/3 f1.5 +... + 1/6 fn)
    #with fct, a, b, and N representing the same things, just the result changes
    delx = (b - a) / N   #rectangle width
    
    result = 0
    for i in np.arange(N):
        x       = a + i * delx
        result += (fct(x) + 4 * fct(x + 0.5 * delx) + fct(x + delx)) / 6 * delx   
    return result

In [6]:
#I'm only doing it with Simpsons rule

for i in range(1,6):
    lower = 0
    upper = np.pi/2
    numb_x = 10**i
    integral = simpson_int(sin_test, lower, upper, numb_x)
    print(f"N = {i};", f"integral = {integral};", f"error = {abs(integral - 1.0)/1.0}")

N = 1; integral = 1.0000002115465914; error = 2.115465913554715e-07
N = 2; integral = 1.0000000000211395; error = 2.113953456728268e-11
N = 3; integral = 1.000000000000002; error = 1.9984014443252818e-15
N = 4; integral = 1.0000000000000004; error = 4.440892098500626e-16
N = 5; integral = 1.0000000000000004; error = 4.440892098500626e-16


For increasing N, the error appears to decreases by a factor of $10^4$; so this means that it is a fourth order method.

**Problem 2:**  

- Having already done a basic transit analysis of a uniform star, we take a star with limb darkening: parameterized with the function $ I(r) $, which is the intensity of the sun’s surface as a function of radius $r$, such that the flux:

\begin{equation}
F(p, z) = \frac{ \int_0^1 I(r) \left[ 1 - \delta(p, r, z) \right] 2r\,dr }{ \int_0^1 I(r) 2r\,dr }
\end{equation}

where

\begin{equation}
\delta(p, r, z) =
\begin{cases}
0 & \text{if } r \geq z + p \text{or } r \leq z - p\\
1 & \text{if } r +z \leq p \\
\frac{1}{\pi} \cos^{-1}\left( \frac{z^2 - p^2 + r^2}{2zr} \right) & \text{otherwise}
\end{cases}
\end{equation}

- Evaluating for $p = 0.2$ and $z = 0.9$.

In [ ]:
#Since the function depends on delta, I write it first

def delta_fct(p, r, z):
    if (r >= z + p) or (r <= z - p):
        return 0
    elif (r + z <= p):
        return 1
    else:
        #Learning to prevent errors, i.e. division by zero
        if (z != 0) and (r != 0):
            arg = (z**2 - p**2 + r**2) / (2 * z * r)
        else:
            print('Either z or r vanished. z =',z,'; and r =',r,'.')
        return (1/np.pi) * np.arccos(arg)

In [8]:
#While we define I(r) as 1, I will still define it

def nolimb(r):
    return 1

In [12]:
#Since we are just doing a serial, not an array

def flux(p, z, limb_fct, integrator, N):
    #Flux depends on p and z
    #But also has the limb_fct which depends on r
    #And we do the intrgration

    def numerator(r, p = p, z = z):
        return limb_fct(r) * (1 - delta_fct(p, r, z)) * 2 * r
    def denominator(r):
        return limb_fct(r) * 2.0 * r
    
    #Bounds
    lower = 0
    upper = 1
    
    top = integrator(numerator, lower, upper, N)
    bottom = integrator(denominator, lower, upper, N)

    return top/bottom

In [ ]:
#For error analysis, from pset 1, we have the lambda fct

def lambda_fct(p, z):
    #Case when the planet is off of the star entirely
    if z > 1 + p:
        return 0

    #Case where the planet is bigger than the star (i.e. eclipse)
    elif z <= p - 1:   
        return 1

    #Case where the planet is entirely on the star 
    elif z <= 1 - p:   
        return p*p       
    
    #Now the big part is for partial obscuration
    elif (m.fabs(1 - p) < z) and (z <= (1 + p)):
        
        root = 0.5 * np.sqrt(4 * z**2 - (1 + z**2 - p**2)**2)
        k0 = (p**2 + z**2 - 1) / (2 * p * z)  
        k1 = (1 - p**2 + z**2) / (2 * z)  
        kappa_0  = m.acos(k0)
        kappa_1  = m.acos(k1)
        
        #print(sqrt_fact, kappa_0, kappa_1)
        return (1 / m.pi) * (p**2 * kappa_0 + kappa_1 - root)

#original flux from problem set 1 for an error analysis
def og_flux(p, z):
    return 1 - lambda_fct(0.2, np.abs(z))

In [ ]:
p = 0.2
z = 0.9

#Comparing to original flux
f_true = og_flux(p, z)

#With simpson integral method
print('N,    f_int,    f_true,     relative error')
for i in range(1,5):
    f_int = flux(p, z, nolimb, simpson_int, 10**i)
    print(10**i, f_int, f_true, abs(f_int - f_true)/f_true)

N,    f_int,    f_true,     relative error
10 0.9687420048514388 0.9684170049114548 0.0003355991668214699
100 0.9684272016510476 0.9684170049114548 1.0529285980192063e-05
1000 0.9684173272303755 0.9684170049114548 3.328307115024195e-07
10000 0.9684170151036687 0.9684170049114548 1.0524612754640314e-08


**Problem 3:**  

- Using a Monte-Carlo integration to evaluate the same integrals as in the last problem (again assuming $I(r) = 1$).
- Generate N random x and y values that are each drawn from a uniform distribution from -1 to 1 and rejecting points that lie outside the unit circle, $N_1$.
- Count number of accepted points, $N_2$.
- Estimate $F(p, z)$:

\begin{equation}
F(p, z) \approx \frac{N_1 - N_2}{N_1}
\end{equation}

In [ ]:
#Creating the monte carlo integration unsing np.random

def mc_planet(p, z, N):
    #since normalized
    radius = 1
    n1 = 0 
    n2 = 0

    for i in np.arange(N):
        xr = random.uniform(-1, 1)
        yr = random.uniform(-1, 1)
        
        #Check points for inclusion
        #Inside the stellar disk
        if xr**2 + yr**2 <= 1:
            n1 += 1.0
            #Also inside the planet
            if (xr - z)**2 + yr**2 < p**2:
                n2 +=1.0
    
    #Check to see if we got enough points to avoid division by zero
    #and return result if so
    if n1 > 0:
        return (n1 - n2)/n1
    else:
        print("Too few points", N)

In [ ]:
#We already have p, z, and f_true from above

print('N, f_int, f_true, relative error')
for i in range(1,5):
    f_int = mc_planet(p, z, 10**i)
    print(10**i, f_int, f_true, abs(f_int - f_true)/f_true)

N,    f_int,    f_true,     relative error
10 1.0 0.9684170049114548 0.03261301167613528
100 0.9230769230769231 0.9684170049114548 0.046818758452798155
1000 0.967948717948718 0.9684170049114548 0.00048355921092033696
10000 0.9700446144040791 0.9684170049114548 0.0016806907400114496


Error bounces A LOT, probably from the randomness.